In [ ]:
cd ..

In [2]:
from datasets.seq2seq_datasets import Seq2SeqIndexedDataset

from models.transformer import Transformer, TransformerEncoder, TransformerDecoder
from trainers.seq2seq_trainer import Seq2SeqTrainer, seq2seq_collate_fn

import torch
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

from utils import get_logger

from dictionaries import BaseDictionary

from trainers.sorted_2d_batch_sampler import Sorted2DBatchSampler, Sorted2DBatchSamplerOnTheFly

In [3]:
dictionary = BaseDictionary.load('base_dictionary')
dictionary.vocabulary_size

48034

In [4]:
device = torch.device('cpu')

In [5]:
embedding = torch.nn.Embedding(num_embeddings=dictionary.vocabulary_size, embedding_dim=512)
encoder = TransformerEncoder(layers_count=3, d_model=512, heads_count=8, d_ff=512, dropout_prob=0.2, embedding=embedding)
decoder = TransformerDecoder(layers_count=3, d_model=512, heads_count=8, d_ff=512, dropout_prob=0.2, embedding=embedding)
model = Transformer(encoder, decoder)

In [6]:
logger = get_logger(log_name='transformer')
train_dataset = Seq2SeqIndexedDataset('train')
val_dataset = Seq2SeqIndexedDataset('val')

trainer = Seq2SeqTrainer(model=model, 
                         train_dataloader=DataLoader(train_dataset, 
                                                     batch_sampler=Sorted2DBatchSamplerOnTheFly(train_dataset, batch_size=32, drop_last=False, max_length=100), 
                                                     collate_fn=seq2seq_collate_fn), 
                         val_dataloader=DataLoader(val_dataset, 
                                                   batch_sampler=Sorted2DBatchSamplerOnTheFly(val_dataset, batch_size=100, drop_last=False, max_length=100),
                                                   collate_fn=seq2seq_collate_fn), 
                         loss_function=CrossEntropyLoss(reduce=False),
                         optimizer=Adam(model.parameters()),
                         device=device,
                         logger=logger)

In [14]:
trainer.run(epochs=10)

  0%|          | 0/817 [00:00<?, ?it/s]


TypeError: forward() takes 3 positional arguments but 4 were given

In [7]:
from generators.base_generators import CandidatesTransformerGenerator

In [10]:
generator = CandidatesTransformerGenerator(model=model,
                    preprocess=lambda source: [dictionary[word] for word in sum([sentence.split() for sentence in source], [])],
                    postprocess=lambda h: ' '.join([dictionary.idx2word[token_id] for token_id in h[:-1]]), # exclude EndSent token
#                     checkpoint_filepath='parameters/Seq2SeqModel/Seq2SeqModel_0_2018-08-05 14:23:30.982203.pth',
                    max_length=30,
                    beam_size=8
                   )

In [11]:
generator.generate_candidates(['버려진 섬마다 꽃이 피었다', '그 꽃들은 정말 아름다운 꽃이었다'], n_candidates=5)

['워낙에 좇을 ⎡부인. 풍경에 ⎡전하! 사냐고 적군들도 문경지교. 누려야지 ⎜오그라들지도 푸르스름한 비통한 얼굴에도 목구멍에서 휘종도 통증에 정계 싱그러웠던 벌이는 펄럭이며 스며들겠구나. 물으면 만났사옵니다. ⎡……참으로 천장에 유씨라는 체통도 치켜뜨며 ⎡고개를',
 '워낙에 좇을 ⎡부인. 풍경에 ⎡전하! 사냐고 적군들도 문경지교. 누려야지 ⎜오그라들지도 푸르스름한 비통한 얼굴에도 목구멍에서 휘종도 통증에 정계 싱그러웠던 벌이는 펄럭이며 스며들겠구나. 물으면 만났사옵니다. ⎡……참으로 천장에 유씨라는 체통도 치켜뜨며 ⎡고개를',
 '워낙에 좇을 ⎡부인. 풍경에 ⎡전하! 사냐고 적군들도 문경지교. 누려야지 ⎜오그라들지도 푸르스름한 비통한 얼굴에도 목구멍에서 휘종도 통증에 정계 싱그러웠던 벌이는 펄럭이며 스며들겠구나. 물으면 만났사옵니다. ⎡……참으로 천장에 유씨라는 체통도 치켜뜨며 ⎡어째서',
 '워낙에 좇을 ⎡부인. 풍경에 ⎡전하! 사냐고 적군들도 문경지교. 누려야지 ⎜오그라들지도 푸르스름한 비통한 얼굴에도 목구멍에서 휘종도 통증에 정계 싱그러웠던 벌이는 펄럭이며 스며들겠구나. 물으면 만났사옵니다. ⎡……참으로 천장에 유씨라는 ⎡막내 돌보지 어떻고요.',
 '워낙에 좇을 ⎡부인. 풍경에 ⎡전하! 사냐고 적군들도 문경지교. 누려야지 ⎜오그라들지도 푸르스름한 비통한 얼굴에도 목구멍에서 휘종도 통증에 정계 싱그러웠던 벌이는 펄럭이며 스며들겠구나. 물으면 만났사옵니다. ⎡……참으로 천장에 유씨라는 ⎡막내 돌보지 어떻고요.']

In [9]:
%debug

> /Users/dreamgonfly/anaconda3/lib/python3.6/site-packages/torch/tensor.py(302)expand_as()
    300 
    301     def expand_as(self, tensor):
--> 302         return self.expand(tensor.size())
    303 
    304     def unique(self, sorted=False, return_inverse=False):



ipdb>  q
